In [22]:
!conda install --yes --quiet pymongo

import pymongo
import re
import requests
import pandas as pd
import numpy as np
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import sys



# All requested packages already installed.
# packages in environment at /opt/conda:
#
pymongo                   3.4.0                    py36_0    defaults


In [ ]:
client = pymongo.MongoClient('54.200.30.7', 27016)

In [49]:
client['business_software']['level_0'].find()

In [55]:
cursor = client.nutrition.level_0.find()
display(list(cursor))

[{'_id': ObjectId('5a204c95e03ab606d6028318'),
  'category': 'nutrition',
  'raw page text': 'Nutrition is the science that interprets the interaction of nutrients and other substances in food in relation to maintenance growth reproduction health and disease of an organism It includes food intake absorption assimilation biosynthesis catabolism and excretion The diet of an organism is what it eats which is largely determined by the availability and palatability of foods For humans a healthy diet includes preparation of food and storage methods that preserve nutrients from oxidation heat or leaching and that reduce risk of foodborne illness In humans an unhealthy diet can cause deficiencyrelated diseases such as blindness anemia scurvy preterm birth stillbirth and cretinism or nutrient excess healththreatening conditions such as obesity and metabolic syndrome and such common chronic systemic diseases as cardiovascular disease diabetes and osteoporosis Undernutrition can lead to the wasti

In [50]:
def search_term(search_term):
    
    list_dbs = []
    
    for names in client.database_names():
        for collections in client[names].collection_names():
            cursor_level = client[names][collections].find()
            db_level = list(cursor_level)
            df_level = pd.DataFrame(db_level)
            list_dbs.append(df_level)
    
    df = pd.concat(list_dbs)
    
    top_5_list = []
    final_df = pd.DataFrame()
    
    tfidf_vectorizer = TfidfVectorizer(min_df = 1, stop_words = 'english')
    initial_terms_encoded = tfidf_vectorizer.fit_transform(df['raw page text'])
    initial_terms_encoded_df = pd.DataFrame(initial_terms_encoded.toarray(), 
                                       index=df['raw page text'], 
                                       columns=tfidf_vectorizer.get_feature_names())
    
    search_terms_encoded = tfidf_vectorizer.transform([search_term])
    search_terms_encoded_df = pd.DataFrame(search_terms_encoded.toarray(), 
                                       index=[search_term], 
                                       columns=tfidf_vectorizer.get_feature_names())
    
    dtm_with_search_term = initial_terms_encoded_df.append(search_terms_encoded_df)
    n_components = 200
    SVD = TruncatedSVD(n_components)
    component_names = ["component_"+str(i+1) for i in range(n_components)]
    
    svd_matrix = SVD.fit_transform(dtm_with_search_term)
    svd_df = pd.DataFrame(svd_matrix, 
                      index=dtm_with_search_term.index, 
                      columns=component_names)
    
    search_term_svd_vector = svd_df.loc[search_terms_encoded_df.index]
    svd_df['cosine_sim'] = cosine_similarity(svd_df, search_term_svd_vector)
    top_5 = svd_df[['cosine_sim']].sort_values('cosine_sim', ascending=False).head(6)
    
    for page in range(1,6):
        top_5_list.append(df[df['raw page text'] == top_5.index[page]]['title'])
    
    final_df['title'] = pd.concat(top_5_list)
    return final_df

In [51]:
q = 0
while q == 0:
    name = input("Search (type 'q' to exit): ")
    if name == 'q':
        q = 1
        pass
    if name != 'q':
        search = search_term(name)
        display(search)

Search (type 'q' to exit): health and nutrition


,title
143,Nutrition Education
9,American Society for Nutrition
114,"Loma Linda University School of Public Health,..."
174,Preventive nutrition
125,MRC Human Nutrition Research


Search (type 'q' to exit): business software


,title
0,Business software
228,Small business software
29,Business suite
72,Dynamic Business Modeling
27,Business rules approach


Search (type 'q' to exit): q
